In [1]:
import torch

In [2]:
import torchvision

In [3]:
import numpy as np
import math

In [4]:
mnist_trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

In [5]:
mnist_testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

In [6]:
mnist_trainset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [7]:
## Parameters:
n_epochs = 3
batch_size_train = 10000
batch_size_test = 500
learning_rate = 0.01
momentum = 0.5
log_interval = 500

In [8]:
train_loader = torch.utils.data.DataLoader(mnist_trainset,batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_testset,batch_size=batch_size_test, shuffle=True)


In [9]:
import papayaclient

In [10]:
class TheModel(torch.nn.Module):

    def __init__(self):
        super(TheModel, self).__init__()

        self.linear1 = torch.nn.Linear(784, 400)
        self.linear2 = torch.nn.Linear(400, 10)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x1 = x.flatten(start_dim = 1)
        return self.linear2(self.relu(self.linear1(x1)))

In [11]:
TheModel

__main__.TheModel

In [12]:
clients = []
for batchno, (ex_data, ex_labels) in enumerate(train_loader):
    clients.append(papayaclient.PapayaClient(dat = ex_data,
                                            labs = ex_labels,
                                            batch_sz = 500,
                                            num_partners = 5,
                                            model_class = TheModel,
                                            loss_fn = torch.nn.CrossEntropyLoss))


In [13]:
## Train the Nodes
num_epochs_total = 100
num_epochs_per_swap = 5
num_times = (num_epochs_total // num_epochs_per_swap)
for i in range(0, num_times):
    for n in clients:
        for j in range(0, num_epochs_per_swap):
            n.model_train_epoch()
    if i > 1 and i < num_times - 1 :
        for n in clients:
            n.select_partners(3)
        for n in clients:
            for i in range(0, 4) :
                n.update_partner_weights()
        n.average_partners()

In [17]:
for c in clients :
    print(c.logs['stringy'][99])

node2185epoch 99 loss 0.16688966751098633
node4813epoch 99 loss 0.19882036745548248
node2458epoch 99 loss 0.21521618962287903
node2694epoch 99 loss 0.32735294103622437
node1727epoch 99 loss 0.26792699098587036
node935epoch 99 loss 0.48465022444725037


In [19]:
accuracies = {}
with torch.no_grad():
    for i in clients :
        accuracies_node = []
        for batchno, (ex_data, ex_labels) in enumerate(test_loader) :
            accuracies_node.append(((i.model.forward(ex_data).argmax(dim = 1) == ex_labels).float().mean()).item())
        accuracies[i.node_id] = np.array(accuracies_node).mean()

In [20]:
accuracies

{2185: 0.928999999165535,
 4813: 0.9284999996423722,
 2458: 0.9268999934196472,
 2694: 0.9267999976873398,
 1727: 0.9251999944448471,
 935: 0.9072000056505203}